# Geo-Special Analysis

## Load extended libraries

In [46]:
import geopandas as gpd
import pandas as pd
import seaborn as sns
import numpy as np
import rasterio
from matplotlib import pyplot as plt
from shapely.geometry import Point
import shapely
from rasterstats import point_query
import branca as bc
import branca.colormap as cm
import ipyleaflet as ipyl
from ipyleaflet import Map, GeoData, basemaps, LayersControl, Popup, basemap_to_tiles, Marker
import contextily as ctx
from shapely.geometry import Point, LineString, Polygon
import folium
from folium.plugins import MarkerCluster, HeatMap
import mapclassify
from IPython.display import display

## Load datasets

### Air quality

In [2]:
air_df = pd.read_excel('../778project/data/airquality/airqualitydownloaddata_2016-2022.xlsx')
air_df

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration
0,Auckland region,Auckland Council,LAWA-100059,Glen Eden,67,-36.922143,174.652001,Glen Eden,"Residential, NES site",PM10,2016-01-01,11.10
1,Auckland region,Auckland Council,LAWA-100059,Glen Eden,67,-36.922143,174.652001,Glen Eden,"Residential, NES site",PM10,2016-01-02,10.60
2,Auckland region,Auckland Council,LAWA-100059,Glen Eden,67,-36.922143,174.652001,Glen Eden,"Residential, NES site",PM10,2016-01-03,4.70
3,Auckland region,Auckland Council,LAWA-100059,Glen Eden,67,-36.922143,174.652001,Glen Eden,"Residential, NES site",PM10,2016-01-04,6.90
4,Auckland region,Auckland Council,LAWA-100059,Glen Eden,67,-36.922143,174.652001,Glen Eden,"Residential, NES site",PM10,2016-01-05,13.20
...,...,...,...,...,...,...,...,...,...,...,...,...
202161,West Coast region,West Coast Regional Council,LAWA-100116,Reefton AQ at School Pool,Reefton Air Quality @ School,-42.114232,171.862887,Reefton,"Residential, NES site",PM10,2022-12-27,5.25
202162,West Coast region,West Coast Regional Council,LAWA-100116,Reefton AQ at School Pool,Reefton Air Quality @ School,-42.114232,171.862887,Reefton,"Residential, NES site",PM10,2022-12-28,8.56
202163,West Coast region,West Coast Regional Council,LAWA-100116,Reefton AQ at School Pool,Reefton Air Quality @ School,-42.114232,171.862887,Reefton,"Residential, NES site",PM10,2022-12-29,11.22
202164,West Coast region,West Coast Regional Council,LAWA-100116,Reefton AQ at School Pool,Reefton Air Quality @ School,-42.114232,171.862887,Reefton,"Residential, NES site",PM10,2022-12-30,6.90


In [21]:
print(air_df.dtypes)
print(air_df.columns)

Region                   object
Agency                   object
LAWA Site ID             object
Site Name                object
Site ID                  object
Latitude                float64
Longitude               float64
Town                     object
Site Type                object
Indicator                object
Sample Date      datetime64[ns]
Concentration           float64
dtype: object
Index(['Region', 'Agency', 'LAWA Site ID', 'Site Name', 'Site ID', 'Latitude',
       'Longitude', 'Town', 'Site Type', 'Indicator', 'Sample Date',
       'Concentration'],
      dtype='object')


Auckland

In [76]:
air_akl = air_df[air_df['Region'] == 'Auckland region']
akl_pm25 = air_akl[air_akl['Indicator'] == 'PM2.5']
akl_pm10 = air_akl[air_akl['Indicator'] == 'PM10']

In [77]:
average = akl_pm25.groupby('Site Name')['Concentration'].mean().reset_index()
df_unique = akl_pm25.drop('Concentration', axis=1).drop_duplicates('Site Name')
akl_pm25 = pd.merge(df_unique, average, on='Site Name')
akl_pm25

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration
0,Auckland region,Auckland Council,LAWA-100106,Patumahoe,2,-37.204504,174.864019,Patumahoe,"Residential, NES site",PM2.5,2016-01-01,4.892389
1,Auckland region,Auckland Council,LAWA-100110,Penrose,7,-36.904558,174.815564,Penrose,"Residential, Industrial, NES site",PM2.5,2016-01-01,6.274326
2,Auckland region,Auckland Council,LAWA-100113,Queen Street,9,-36.847653,174.765577,Auckland City Centre,"Traffic, NES site",PM2.5,2017-07-01,7.182304
3,Auckland region,Auckland Council,LAWA-100124,Takapuna,23,-36.780305,174.748904,Takapuna,"Traffic, NES site",PM2.5,2016-01-01,6.542803


In [78]:
geometry = [Point(xy) for xy in zip(akl_pm25['Longitude'], akl_pm25['Latitude'])]
akl_gdf25 = gpd.GeoDataFrame(akl_pm25, geometry=geometry)
akl_gdf25

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration,geometry
0,Auckland region,Auckland Council,LAWA-100106,Patumahoe,2,-37.204504,174.864019,Patumahoe,"Residential, NES site",PM2.5,2016-01-01,4.892389,POINT (174.86402 -37.20450)
1,Auckland region,Auckland Council,LAWA-100110,Penrose,7,-36.904558,174.815564,Penrose,"Residential, Industrial, NES site",PM2.5,2016-01-01,6.274326,POINT (174.81556 -36.90456)
2,Auckland region,Auckland Council,LAWA-100113,Queen Street,9,-36.847653,174.765577,Auckland City Centre,"Traffic, NES site",PM2.5,2017-07-01,7.182304,POINT (174.76558 -36.84765)
3,Auckland region,Auckland Council,LAWA-100124,Takapuna,23,-36.780305,174.748904,Takapuna,"Traffic, NES site",PM2.5,2016-01-01,6.542803,POINT (174.74890 -36.78031)


In [79]:
average = akl_pm10.groupby('Site Name')['Concentration'].mean().reset_index()
df_unique = akl_pm10.drop('Concentration', axis=1).drop_duplicates('Site Name')
akl_pm10 = pd.merge(df_unique, average, on='Site Name')
akl_pm10

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration
0,Auckland region,Auckland Council,LAWA-100059,Glen Eden,67,-36.922143,174.652001,Glen Eden,"Residential, NES site",PM10,2016-01-01,12.454259
1,Auckland region,Auckland Council,LAWA-100064,Henderson,5,-36.868000,174.628353,Henderson,"Residential, Traffic, NES site",PM10,2016-01-01,12.126523
2,Auckland region,Auckland Council,LAWA-100103,Pakuranga,1,-36.905484,174.890289,Pakuranga,"Residential, NES site",PM10,2016-01-01,12.573739
3,Auckland region,Auckland Council,LAWA-100106,Patumahoe,2,-37.204504,174.864019,Patumahoe,"Residential, NES site",PM10,2016-01-01,11.928860
4,Auckland region,Auckland Council,LAWA-100110,Penrose,7,-36.904558,174.815564,Penrose,"Residential, Industrial, NES site",PM10,2016-01-01,14.681999
5,Auckland region,Auckland Council,LAWA-100113,Queen Street,9,-36.847653,174.765577,Auckland City Centre,"Traffic, NES site",PM10,2017-03-04,17.300244
6,Auckland region,Auckland Council,LAWA-100124,Takapuna,23,-36.780305,174.748904,Takapuna,"Traffic, NES site",PM10,2016-01-01,12.904793


In [80]:
geometry = [Point(xy) for xy in zip(akl_pm10['Longitude'], akl_pm10['Latitude'])]
akl_gdf10 = gpd.GeoDataFrame(akl_pm10, geometry=geometry)
akl_gdf10

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration,geometry
0,Auckland region,Auckland Council,LAWA-100059,Glen Eden,67,-36.922143,174.652001,Glen Eden,"Residential, NES site",PM10,2016-01-01,12.454259,POINT (174.65200 -36.92214)
1,Auckland region,Auckland Council,LAWA-100064,Henderson,5,-36.868000,174.628353,Henderson,"Residential, Traffic, NES site",PM10,2016-01-01,12.126523,POINT (174.62835 -36.86800)
2,Auckland region,Auckland Council,LAWA-100103,Pakuranga,1,-36.905484,174.890289,Pakuranga,"Residential, NES site",PM10,2016-01-01,12.573739,POINT (174.89029 -36.90548)
3,Auckland region,Auckland Council,LAWA-100106,Patumahoe,2,-37.204504,174.864019,Patumahoe,"Residential, NES site",PM10,2016-01-01,11.928860,POINT (174.86402 -37.20450)
4,Auckland region,Auckland Council,LAWA-100110,Penrose,7,-36.904558,174.815564,Penrose,"Residential, Industrial, NES site",PM10,2016-01-01,14.681999,POINT (174.81556 -36.90456)
5,Auckland region,Auckland Council,LAWA-100113,Queen Street,9,-36.847653,174.765577,Auckland City Centre,"Traffic, NES site",PM10,2017-03-04,17.300244,POINT (174.76558 -36.84765)
6,Auckland region,Auckland Council,LAWA-100124,Takapuna,23,-36.780305,174.748904,Takapuna,"Traffic, NES site",PM10,2016-01-01,12.904793,POINT (174.74890 -36.78031)


Wellington

In [71]:
air_wlg = air_df[air_df['Region'] == 'Wellington Region']
wlg_pm25 = air_wlg[air_wlg['Indicator'] == 'PM2.5']
wlg_pm10 = air_wlg[air_wlg['Indicator'] == 'PM10']

In [72]:
average = wlg_pm25.groupby('Site Name')['Concentration'].mean().reset_index()
df_unique = wlg_pm25.drop('Concentration', axis=1).drop_duplicates('Site Name')
wlg_pm25 = pd.merge(df_unique, average, on='Site Name')
wlg_pm25

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration
0,Wellington Region,Greater Wellington Regional Council,LAWA-100083,Masterton East (Herbert Street),Chanel College AQ,-40.959198,175.653211,Masterton,"Residential, NES site",PM2.5,2016-01-01 01:00:00,10.913698
1,Wellington Region,Greater Wellington Regional Council,LAWA-100084,Masterton West (Pownall Street),Wairarapa College AQ,-40.952307,175.646614,Masterton,"Residential, NES site",PM2.5,2016-01-01 01:00:00,9.722611
2,Wellington Region,Greater Wellington Regional Council,LAWA-100136,Wainuiomata (Moohan Street),Wainuiomata Bowling Club AQ,-41.267711,174.953751,Wainuiomata,"Residential, NES site",PM2.5,2016-01-01 01:00:00,5.804572
3,Wellington Region,Greater Wellington Regional Council,LAWA-100142,Wellington Central (Willis Street),Willis St AQ,-41.293622,174.771918,Inner City Wellington,"Traffic, NES site",PM2.5,2016-01-09 01:00:00,5.496855


In [73]:
geometry = [Point(xy) for xy in zip(wlg_pm25['Longitude'], wlg_pm25['Latitude'])]
wlg_gdf25 = gpd.GeoDataFrame(wlg_pm25, geometry=geometry)
wlg_gdf25

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration,geometry
0,Wellington Region,Greater Wellington Regional Council,LAWA-100083,Masterton East (Herbert Street),Chanel College AQ,-40.959198,175.653211,Masterton,"Residential, NES site",PM2.5,2016-01-01 01:00:00,10.913698,POINT (175.65321 -40.95920)
1,Wellington Region,Greater Wellington Regional Council,LAWA-100084,Masterton West (Pownall Street),Wairarapa College AQ,-40.952307,175.646614,Masterton,"Residential, NES site",PM2.5,2016-01-01 01:00:00,9.722611,POINT (175.64661 -40.95231)
2,Wellington Region,Greater Wellington Regional Council,LAWA-100136,Wainuiomata (Moohan Street),Wainuiomata Bowling Club AQ,-41.267711,174.953751,Wainuiomata,"Residential, NES site",PM2.5,2016-01-01 01:00:00,5.804572,POINT (174.95375 -41.26771)
3,Wellington Region,Greater Wellington Regional Council,LAWA-100142,Wellington Central (Willis Street),Willis St AQ,-41.293622,174.771918,Inner City Wellington,"Traffic, NES site",PM2.5,2016-01-09 01:00:00,5.496855,POINT (174.77192 -41.29362)


In [74]:
average = wlg_pm10.groupby('Site Name')['Concentration'].mean().reset_index()
df_unique = wlg_pm10.drop('Concentration', axis=1).drop_duplicates('Site Name')
wlg_pm10 = pd.merge(df_unique, average, on='Site Name')
wlg_pm10

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration
0,Wellington Region,Greater Wellington Regional Council,LAWA-100080,Lower Hutt (Phil Evans Reserve),Birch Lane AQ,-41.212607,174.920876,Central Lower Hutt,"Residential, NES site",PM10,2016-01-01 01:00:00,11.033467
1,Wellington Region,Greater Wellington Regional Council,LAWA-100083,Masterton East (Herbert Street),Chanel College AQ,-40.959198,175.653211,Masterton,"Residential, NES site",PM10,2016-01-01 01:00:00,14.495449
2,Wellington Region,Greater Wellington Regional Council,LAWA-100084,Masterton West (Pownall Street),Wairarapa College AQ,-40.952307,175.646614,Masterton,"Residential, NES site",PM10,2016-01-01 01:00:00,13.892114
3,Wellington Region,Greater Wellington Regional Council,LAWA-100132,Upper Hutt (Savage Park),Upper Hutt at Savage Park AQ,-41.121543,175.070345,Central Upper Hutt,"Residential, NES site",PM10,2016-01-01 01:00:00,10.291112
4,Wellington Region,Greater Wellington Regional Council,LAWA-100136,Wainuiomata (Moohan Street),Wainuiomata Bowling Club AQ,-41.267711,174.953751,Wainuiomata,"Residential, NES site",PM10,2016-01-01 01:00:00,10.758411
5,Wellington Region,Greater Wellington Regional Council,LAWA-100142,Wellington Central (Willis Street),Willis St AQ,-41.293622,174.771918,Inner City Wellington,"Traffic, NES site",PM10,2016-01-09 01:00:00,12.356307


In [75]:
geometry = [Point(xy) for xy in zip(wlg_pm10['Longitude'], wlg_pm10['Latitude'])]
wlg_gdf10 = gpd.GeoDataFrame(wlg_pm10, geometry=geometry)
wlg_gdf10

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration,geometry
0,Wellington Region,Greater Wellington Regional Council,LAWA-100080,Lower Hutt (Phil Evans Reserve),Birch Lane AQ,-41.212607,174.920876,Central Lower Hutt,"Residential, NES site",PM10,2016-01-01 01:00:00,11.033467,POINT (174.92088 -41.21261)
1,Wellington Region,Greater Wellington Regional Council,LAWA-100083,Masterton East (Herbert Street),Chanel College AQ,-40.959198,175.653211,Masterton,"Residential, NES site",PM10,2016-01-01 01:00:00,14.495449,POINT (175.65321 -40.95920)
2,Wellington Region,Greater Wellington Regional Council,LAWA-100084,Masterton West (Pownall Street),Wairarapa College AQ,-40.952307,175.646614,Masterton,"Residential, NES site",PM10,2016-01-01 01:00:00,13.892114,POINT (175.64661 -40.95231)
3,Wellington Region,Greater Wellington Regional Council,LAWA-100132,Upper Hutt (Savage Park),Upper Hutt at Savage Park AQ,-41.121543,175.070345,Central Upper Hutt,"Residential, NES site",PM10,2016-01-01 01:00:00,10.291112,POINT (175.07035 -41.12154)
4,Wellington Region,Greater Wellington Regional Council,LAWA-100136,Wainuiomata (Moohan Street),Wainuiomata Bowling Club AQ,-41.267711,174.953751,Wainuiomata,"Residential, NES site",PM10,2016-01-01 01:00:00,10.758411,POINT (174.95375 -41.26771)
5,Wellington Region,Greater Wellington Regional Council,LAWA-100142,Wellington Central (Willis Street),Willis St AQ,-41.293622,174.771918,Inner City Wellington,"Traffic, NES site",PM10,2016-01-09 01:00:00,12.356307,POINT (174.77192 -41.29362)


Christchurch

In [66]:
air_chc = air_df[air_df['Region'] == 'Canterbury region']
chc_pm25 = air_chc[air_chc['Indicator'] == 'PM2.5']
chc_pm10 = air_chc[air_chc['Indicator'] == 'PM10']

In [67]:
average = chc_pm25.groupby('Site Name')['Concentration'].mean().reset_index()
df_unique = chc_pm25.drop('Concentration', axis=1).drop_duplicates('Site Name')
chc_pm25 = pd.merge(df_unique, average, on='Site Name')
chc_pm25

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration
0,Canterbury region,Environment Canterbury,LAWA-100164,Anzac Square,Timaru Anzac Square,-44.404483,171.249663,Timaru,"Residential, NES site",PM2.5,2016-01-01 01:00:00,11.120141
1,Canterbury region,Environment Canterbury,LAWA-100056,Geraldine,Geraldine,-44.100181,171.241481,Geraldine,"Residential, NES site",PM2.5,2016-02-19 01:00:00,9.399443
2,Canterbury region,Environment Canterbury,LAWA-100025,Hampstead,Ashburton,-43.912238,171.755236,Ashburton,"Residential, NES site",PM2.5,2016-01-01 01:00:00,8.812500
3,Canterbury region,Environment Canterbury,LAWA-100071,Kaiapoi,Kaiapoi,-43.384643,172.651991,Kaiapoi,"Residential, NES site",PM2.5,2016-09-04 00:00:00,10.212357
4,Canterbury region,Environment Canterbury,LAWA-100115,Rangiora,Rangiora,-43.307540,172.595265,Rangiora,"Residential, NES site",PM2.5,2016-01-01 01:00:00,8.965595
5,Canterbury region,Environment Canterbury,LAWA-100122,St Albans,St Albans,-43.511257,172.633705,Christchurch,Residential,PM2.5,2016-01-01 01:00:00,9.029405
6,Canterbury region,Environment Canterbury,LAWA-102877,St Albans EP,St Albans EP,-43.508536,172.635926,Christchurch,"Residential, NES site",PM2.5,2021-12-03 01:00:00,7.859381
7,Canterbury region,Environment Canterbury,LAWA-100159,Waimate Kennedy,Waimate Kennedy,-44.732601,171.049930,Waimate,"Residential, NES site",PM2.5,2016-01-01 01:00:00,8.409265
8,Canterbury region,Environment Canterbury,LAWA-102266,Washdyke Alpine,Washdyke Alpine,-44.356199,171.242334,Timaru,"Industrial, NES site",PM2.5,2019-03-19 01:00:00,7.703564
9,Canterbury region,Environment Canterbury,LAWA-100146,Woolston,Woolston,-43.557158,172.681276,Christchurch,"Residential, Industrial, NES site",PM2.5,2016-01-01 01:00:00,6.752920


In [68]:
geometry = [Point(xy) for xy in zip(chc_pm25['Longitude'], chc_pm25['Latitude'])]
chc_gdf25 = gpd.GeoDataFrame(chc_pm25, geometry=geometry)
chc_gdf25

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration,geometry
0,Canterbury region,Environment Canterbury,LAWA-100164,Anzac Square,Timaru Anzac Square,-44.404483,171.249663,Timaru,"Residential, NES site",PM2.5,2016-01-01 01:00:00,11.120141,POINT (171.24966 -44.40448)
1,Canterbury region,Environment Canterbury,LAWA-100056,Geraldine,Geraldine,-44.100181,171.241481,Geraldine,"Residential, NES site",PM2.5,2016-02-19 01:00:00,9.399443,POINT (171.24148 -44.10018)
2,Canterbury region,Environment Canterbury,LAWA-100025,Hampstead,Ashburton,-43.912238,171.755236,Ashburton,"Residential, NES site",PM2.5,2016-01-01 01:00:00,8.812500,POINT (171.75524 -43.91224)
3,Canterbury region,Environment Canterbury,LAWA-100071,Kaiapoi,Kaiapoi,-43.384643,172.651991,Kaiapoi,"Residential, NES site",PM2.5,2016-09-04 00:00:00,10.212357,POINT (172.65199 -43.38464)
4,Canterbury region,Environment Canterbury,LAWA-100115,Rangiora,Rangiora,-43.307540,172.595265,Rangiora,"Residential, NES site",PM2.5,2016-01-01 01:00:00,8.965595,POINT (172.59526 -43.30754)
5,Canterbury region,Environment Canterbury,LAWA-100122,St Albans,St Albans,-43.511257,172.633705,Christchurch,Residential,PM2.5,2016-01-01 01:00:00,9.029405,POINT (172.63371 -43.51126)
6,Canterbury region,Environment Canterbury,LAWA-102877,St Albans EP,St Albans EP,-43.508536,172.635926,Christchurch,"Residential, NES site",PM2.5,2021-12-03 01:00:00,7.859381,POINT (172.63593 -43.50854)
7,Canterbury region,Environment Canterbury,LAWA-100159,Waimate Kennedy,Waimate Kennedy,-44.732601,171.049930,Waimate,"Residential, NES site",PM2.5,2016-01-01 01:00:00,8.409265,POINT (171.04993 -44.73260)
8,Canterbury region,Environment Canterbury,LAWA-102266,Washdyke Alpine,Washdyke Alpine,-44.356199,171.242334,Timaru,"Industrial, NES site",PM2.5,2019-03-19 01:00:00,7.703564,POINT (171.24233 -44.35620)
9,Canterbury region,Environment Canterbury,LAWA-100146,Woolston,Woolston,-43.557158,172.681276,Christchurch,"Residential, Industrial, NES site",PM2.5,2016-01-01 01:00:00,6.752920,POINT (172.68128 -43.55716)


In [69]:
average = chc_pm10.groupby('Site Name')['Concentration'].mean().reset_index()
df_unique = chc_pm10.drop('Concentration', axis=1).drop_duplicates('Site Name')
chc_pm10 = pd.merge(df_unique, average, on='Site Name')
chc_pm10

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration
0,Canterbury region,Environment Canterbury,LAWA-100164,Anzac Square,Timaru Anzac Square,-44.404483,171.249663,Timaru,"Residential, NES site",PM10,2016-01-01 01:00:00,21.319804
1,Canterbury region,Environment Canterbury,LAWA-100056,Geraldine,Geraldine,-44.100181,171.241481,Geraldine,"Residential, NES site",PM10,2016-01-01 01:00:00,16.056248
2,Canterbury region,Environment Canterbury,LAWA-100025,Hampstead,Ashburton,-43.912238,171.755236,Ashburton,"Residential, NES site",PM10,2016-01-01 01:00:00,16.222323
3,Canterbury region,Environment Canterbury,LAWA-100071,Kaiapoi,Kaiapoi,-43.384643,172.651991,Kaiapoi,"Residential, NES site",PM10,2016-01-01 01:00:00,17.278510
4,Canterbury region,Environment Canterbury,LAWA-100115,Rangiora,Rangiora,-43.307540,172.595265,Rangiora,"Residential, NES site",PM10,2016-01-01 01:00:00,15.794805
5,Canterbury region,Environment Canterbury,LAWA-100122,St Albans,St Albans,-43.511257,172.633705,Christchurch,Residential,PM10,2016-01-01 01:00:00,17.420336
6,Canterbury region,Environment Canterbury,LAWA-102877,St Albans EP,St Albans EP,-43.508536,172.635926,Christchurch,"Residential, NES site",PM10,2021-12-03 01:00:00,13.242949
7,Canterbury region,Environment Canterbury,LAWA-100159,Waimate Kennedy,Waimate Kennedy,-44.732601,171.049930,Waimate,"Residential, NES site",PM10,2016-01-01 01:00:00,14.938711
8,Canterbury region,Environment Canterbury,LAWA-102266,Washdyke Alpine,Washdyke Alpine,-44.356199,171.242334,Timaru,"Industrial, NES site",PM10,2019-03-19 01:00:00,25.963354
9,Canterbury region,Environment Canterbury,LAWA-100146,Woolston,Woolston,-43.557158,172.681276,Christchurch,"Residential, Industrial, NES site",PM10,2016-01-01 01:00:00,17.734183


In [70]:
geometry = [Point(xy) for xy in zip(chc_pm10['Longitude'], chc_pm10['Latitude'])]
chc_gdf10 = gpd.GeoDataFrame(chc_pm10, geometry=geometry)
chc_gdf10

,Region,Agency,LAWA Site ID,Site Name,Site ID,Latitude,Longitude,Town,Site Type,Indicator,Sample Date,Concentration,geometry
0,Canterbury region,Environment Canterbury,LAWA-100164,Anzac Square,Timaru Anzac Square,-44.404483,171.249663,Timaru,"Residential, NES site",PM10,2016-01-01 01:00:00,21.319804,POINT (171.24966 -44.40448)
1,Canterbury region,Environment Canterbury,LAWA-100056,Geraldine,Geraldine,-44.100181,171.241481,Geraldine,"Residential, NES site",PM10,2016-01-01 01:00:00,16.056248,POINT (171.24148 -44.10018)
2,Canterbury region,Environment Canterbury,LAWA-100025,Hampstead,Ashburton,-43.912238,171.755236,Ashburton,"Residential, NES site",PM10,2016-01-01 01:00:00,16.222323,POINT (171.75524 -43.91224)
3,Canterbury region,Environment Canterbury,LAWA-100071,Kaiapoi,Kaiapoi,-43.384643,172.651991,Kaiapoi,"Residential, NES site",PM10,2016-01-01 01:00:00,17.278510,POINT (172.65199 -43.38464)
4,Canterbury region,Environment Canterbury,LAWA-100115,Rangiora,Rangiora,-43.307540,172.595265,Rangiora,"Residential, NES site",PM10,2016-01-01 01:00:00,15.794805,POINT (172.59526 -43.30754)
5,Canterbury region,Environment Canterbury,LAWA-100122,St Albans,St Albans,-43.511257,172.633705,Christchurch,Residential,PM10,2016-01-01 01:00:00,17.420336,POINT (172.63371 -43.51126)
6,Canterbury region,Environment Canterbury,LAWA-102877,St Albans EP,St Albans EP,-43.508536,172.635926,Christchurch,"Residential, NES site",PM10,2021-12-03 01:00:00,13.242949,POINT (172.63593 -43.50854)
7,Canterbury region,Environment Canterbury,LAWA-100159,Waimate Kennedy,Waimate Kennedy,-44.732601,171.049930,Waimate,"Residential, NES site",PM10,2016-01-01 01:00:00,14.938711,POINT (171.04993 -44.73260)
8,Canterbury region,Environment Canterbury,LAWA-102266,Washdyke Alpine,Washdyke Alpine,-44.356199,171.242334,Timaru,"Industrial, NES site",PM10,2019-03-19 01:00:00,25.963354,POINT (171.24233 -44.35620)
9,Canterbury region,Environment Canterbury,LAWA-100146,Woolston,Woolston,-43.557158,172.681276,Christchurch,"Residential, Industrial, NES site",PM10,2016-01-01 01:00:00,17.734183,POINT (172.68128 -43.55716)


### Traffic

In [15]:
traffic_df = pd.read_csv('../778project/data/traffic/State_highway_traffic_monitoring_sites.csv')
traffic_df

,X,Y,OBJECTID,SH,RS,RP,siteRef,lane,type,percentHeavy,equipmentCurrent,description,region,acceptedDays,AADT5yearsAgo,AADT4yearsAgo,AADT3yearsAgo,AADT2yearsAgo,AADT1yearAgo,siteType
0,1.687583e+06,6.092785e+06,1,10,0,7.38,01000007,Both,Non-Continuous,7.8,Dual Loop,Sth of Puketona Rd (SH11),01 - Northland,27.0,5401.0,5501.0,4954.0,5376.0,5809.0,Regional Non-Continuous
1,1.686763e+06,6.095528e+06,2,10,7,3.01,01000011,Both,Non-Continuous,6.3,Dual Loop,Nth of Wakelin Rd (about 3.5km north of SH11),01 - Northland,28.0,9737.0,9952.0,9001.0,9761.0,10196.0,Regional Non-Continuous
2,1.683980e+06,6.099669e+06,3,10,7,8.23,01000015,Both,Non-Continuous,6.9,Dual Loop,Springbank Road 1km south of Waimate Nth Rd,01 - Northland,35.0,7201.0,7918.0,7419.0,7971.0,8017.0,Regional Non-Continuous
3,1.680411e+06,6.110299e+06,4,10,17,11.14,01000029,Both,Continuous,7.0,Dual Loop,About 1km south of Takou Bay Rd,01 - Northland,340.0,4704.0,4934.0,4853.0,4754.0,5117.0,Regional Continuous
4,1.658993e+06,6.124062e+06,5,10,48,11.93,01000060,Both,Non-Continuous,8.7,Dual Loop,About 1.1 km north of Salvation Rd,01 - Northland,21.0,2458.0,2621.0,2913.0,2981.0,2974.0,Regional Non-Continuous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,1.234794e+06,4.859231e+06,2038,99,0,7.80,09900008,Both,Non-Continuous,7.8,Dual Loop,East of Argyle Corner,14 - Southland,28.0,4440.0,4487.0,4250.0,4647.0,4396.0,Regional Non-Continuous
2038,1.219018e+06,4.858686e+06,2039,99,15,10.14,09900025,Both,Continuous,6.1,Dual Loop,Riverton-near racecourse,14 - Southland,363.0,2487.0,2512.0,2347.0,2481.0,2424.0,Regional Continuous
2039,1.208668e+06,4.853922e+06,2040,99,31,7.97,09900039,Both,Non-Continuous,11.5,Single Loop,Colac,14 - Southland,28.0,1002.0,990.0,967.0,1074.0,1048.0,Regional Non-Continuous
2040,1.189687e+06,4.874773e+06,2041,99,60,15.06,09900075,Both,Non-Continuous,17.1,Single Loop,Tuatapere,14 - Southland,28.0,539.0,526.0,548.0,504.0,520.0,Regional Non-Continuous


In [20]:
print(traffic_df.dtypes)
print(traffic_df.columns)

X                   float64
Y                   float64
OBJECTID              int64
SH                   object
RS                    int64
RP                  float64
siteRef              object
lane                 object
type                 object
percentHeavy        float64
equipmentCurrent     object
description          object
region               object
acceptedDays        float64
AADT5yearsAgo       float64
AADT4yearsAgo       float64
AADT3yearsAgo       float64
AADT2yearsAgo       float64
AADT1yearAgo        float64
siteType             object
dtype: object
Index(['X', 'Y', 'OBJECTID', 'SH', 'RS', 'RP', 'siteRef', 'lane', 'type',
       'percentHeavy', 'equipmentCurrent', 'description', 'region',
       'acceptedDays', 'AADT5yearsAgo', 'AADT4yearsAgo', 'AADT3yearsAgo',
       'AADT2yearsAgo', 'AADT1yearAgo', 'siteType'],
      dtype='object')


Auckland

In [96]:
tr_akl = traffic_df[traffic_df['region'] == '02 - Auckland']
tr_akl['Average AADT'] = tr_akl[['AADT5yearsAgo', 'AADT4yearsAgo', 'AADT3yearsAgo', 'AADT2yearsAgo', 'AADT1yearAgo']].mean(axis=1)
median_average_aadt = tr_akl['Average AADT'].median()
tr_akl['Average AADT'] = tr_akl['Average AADT'].fillna(median_average_aadt)
tr_akl

C:\Users\Alvin Zhao\AppData\Local\Temp\ipykernel_4064\2643576974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_akl['Average AADT'] = tr_akl[['AADT5yearsAgo', 'AADT4yearsAgo', 'AADT3yearsAgo', 'AADT2yearsAgo', 'AADT1yearAgo']].mean(axis=1)
C:\Users\Alvin Zhao\AppData\Local\Temp\ipykernel_4064\2643576974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_akl['Average AADT'] = tr_akl['Average AADT'].fillna(median_average_aadt)


,X,Y,OBJECTID,SH,RS,RP,siteRef,lane,type,percentHeavy,...,description,region,acceptedDays,AADT5yearsAgo,AADT4yearsAgo,AADT3yearsAgo,AADT2yearsAgo,AADT1yearAgo,siteType,Average AADT
83,1.757752e+06,5.919671e+06,84,16,0,0.70,01600001,Both,Virtual,8.5,...,SH16 Stanley St East of Alten Rd - Virtual,02 - Auckland,NaN,51625.0,52420.0,43954.0,42841.0,47563.0,Virtual,47680.6
84,1.741079e+06,5.927503e+06,85,16,19,0.61,01600023,Both,Continuous,4.0,...,SH16 Sth of Coatesville Riverhead Highway,02 - Auckland,178.0,36606.0,37026.0,34550.0,34800.0,37768.0,Regional Continuous,36150.0
85,1.740828e+06,5.927551e+06,86,16,19,1.71,01600024,Both,Continuous,4.3,...,SH16 Nth of Coatesville Riverhead Highway,02 - Auckland,262.0,28785.0,29026.0,27025.0,27218.0,29177.0,Regional Continuous,28246.2
86,1.738500e+06,5.929019e+06,87,16,19,5.37,01600027,Both,Continuous,3.1,...,SH16 Nth of Access Rd,02 - Auckland,355.0,23300.0,22901.0,20660.0,20844.0,22734.0,Regional Continuous,22087.8
87,1.735754e+06,5.929897e+06,88,16,19,7.96,01600031,Both,Continuous,4.9,...,SH16 Sth of Matua Rd (Nth),02 - Auckland,365.0,15568.0,15551.0,13868.0,14348.0,15167.0,Regional Continuous,14900.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,1.772607e+06,5.891053e+06,666,22,0,0.83,02200001,Both,Continuous,7.1,...,SH22 East of Great Sth Rd,02 - Auckland,346.0,26124.0,27207.0,24948.0,24200.0,24905.0,Regional Continuous,25476.8
666,1.771042e+06,5.889939e+06,667,22,0,3.16,02200003,Both,Continuous,5.8,...,SH22 West of Oira Rd,02 - Auckland,365.0,20953.0,21987.0,21358.0,20446.0,19017.0,Regional Continuous,20752.2
667,1.767307e+06,5.887763e+06,668,22,0,8.01,02200008,Both,Continuous,3.9,...,SH22 West of Glenbrook Rd,02 - Auckland,361.0,16474.0,17466.0,17818.0,15445.0,14175.0,Regional Continuous,16275.6
668,1.768218e+06,5.885040e+06,669,22,0,11.52,02200011,Both,Continuous,3.9,...,SH22 East of Heights Rd,02 - Auckland,334.0,16836.0,18636.0,16112.0,14125.0,15658.0,Regional Continuous,16273.4


In [103]:
geometry = [Point(xy) for xy in zip(tr_akl['X'], tr_akl['Y'])]
akl_gdf_tr = gpd.GeoDataFrame(tr_akl, geometry=geometry)
akl_gdf_tr = akl_gdf_tr.set_crs(epsg=2193)
akl_gdf_tr = akl_gdf_tr.to_crs(epsg=4326)
akl_gdf_tr

,X,Y,OBJECTID,SH,RS,RP,siteRef,lane,type,percentHeavy,...,region,acceptedDays,AADT5yearsAgo,AADT4yearsAgo,AADT3yearsAgo,AADT2yearsAgo,AADT1yearAgo,siteType,Average AADT,geometry
83,1.757752e+06,5.919671e+06,84,16,0,0.70,01600001,Both,Virtual,8.5,...,02 - Auckland,NaN,51625.0,52420.0,43954.0,42841.0,47563.0,Virtual,47680.6,POINT (174.76955 -36.85572)
84,1.741079e+06,5.927503e+06,85,16,19,0.61,01600023,Both,Continuous,4.0,...,02 - Auckland,178.0,36606.0,37026.0,34550.0,34800.0,37768.0,Regional Continuous,36150.0,POINT (174.58114 -36.78779)
85,1.740828e+06,5.927551e+06,86,16,19,1.71,01600024,Both,Continuous,4.3,...,02 - Auckland,262.0,28785.0,29026.0,27025.0,27218.0,29177.0,Regional Continuous,28246.2,POINT (174.57832 -36.78739)
86,1.738500e+06,5.929019e+06,87,16,19,5.37,01600027,Both,Continuous,3.1,...,02 - Auckland,355.0,23300.0,22901.0,20660.0,20844.0,22734.0,Regional Continuous,22087.8,POINT (174.55197 -36.77450)
87,1.735754e+06,5.929897e+06,88,16,19,7.96,01600031,Both,Continuous,4.9,...,02 - Auckland,365.0,15568.0,15551.0,13868.0,14348.0,15167.0,Regional Continuous,14900.4,POINT (174.52106 -36.76699)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,1.772607e+06,5.891053e+06,666,22,0,0.83,02200001,Both,Continuous,7.1,...,02 - Auckland,346.0,26124.0,27207.0,24948.0,24200.0,24905.0,Regional Continuous,25476.8,POINT (174.94265 -37.11095)
666,1.771042e+06,5.889939e+06,667,22,0,3.16,02200003,Both,Continuous,5.8,...,02 - Auckland,365.0,20953.0,21987.0,21358.0,20446.0,19017.0,Regional Continuous,20752.2,POINT (174.92530 -37.12128)
667,1.767307e+06,5.887763e+06,668,22,0,8.01,02200008,Both,Continuous,3.9,...,02 - Auckland,361.0,16474.0,17466.0,17818.0,15445.0,14175.0,Regional Continuous,16275.6,POINT (174.88376 -37.14156)
668,1.768218e+06,5.885040e+06,669,22,0,11.52,02200011,Both,Continuous,3.9,...,02 - Auckland,334.0,16836.0,18636.0,16112.0,14125.0,15658.0,Regional Continuous,16273.4,POINT (174.89463 -37.16593)


Wellington

In [98]:
tr_wlg = traffic_df[traffic_df['region'] == '09 - Wellington']
tr_wlg['Average AADT'] = tr_wlg[['AADT5yearsAgo', 'AADT4yearsAgo', 'AADT3yearsAgo', 'AADT2yearsAgo', 'AADT1yearAgo']].mean(axis=1)
median_average_aadt = tr_wlg['Average AADT'].median()
tr_wlg['Average AADT'] = tr_wlg['Average AADT'].fillna(median_average_aadt)
tr_wlg

C:\Users\Alvin Zhao\AppData\Local\Temp\ipykernel_4064\4103606005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_wlg['Average AADT'] = tr_wlg[['AADT5yearsAgo', 'AADT4yearsAgo', 'AADT3yearsAgo', 'AADT2yearsAgo', 'AADT1yearAgo']].mean(axis=1)
C:\Users\Alvin Zhao\AppData\Local\Temp\ipykernel_4064\4103606005.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_wlg['Average AADT'] = tr_wlg['Average AADT'].fillna(median_average_aadt)


,X,Y,OBJECTID,SH,RS,RP,siteRef,lane,type,percentHeavy,...,description,region,acceptedDays,AADT5yearsAgo,AADT4yearsAgo,AADT3yearsAgo,AADT2yearsAgo,AADT1yearAgo,siteType,Average AADT
1248,1.776014e+06,5.477040e+06,1249,1K,1011,0.450,01K11011,Inc,Continuous,0.2,...,Peka Peka Int Mainline Southbound,09 - Wellington,21.0,8023.0,7885.0,7438.0,7578.0,941.0,Regional Continuous,6373.000000
1249,1.771258e+06,5.473631e+06,1250,1K,1011,6.500,01K11017,Inc,Continuous,9.7,...,Te Moana Int Mainline Southbound,09 - Wellington,204.0,6772.0,7136.0,6880.0,4.0,7400.0,Regional Continuous,5638.400000
1250,1.770671e+06,5.472273e+06,1251,1K,1011,8.000,01K11019,Inc,Virtual,6.8,...,Kapiti Expressway Virtual Sth of Waikanae Sout...,09 - Wellington,NaN,NaN,NaN,10725.0,11748.0,12482.0,Virtual,11651.666667
1251,1.768252e+06,5.469309e+06,1252,1K,1011,11.920,01K11023,Inc,Continuous,9.9,...,Kapiti Rd Int Mainline Southbound,09 - Wellington,332.0,6312.0,6554.0,5821.0,6366.0,6849.0,Regional Continuous,6380.400000
1252,1.767713e+06,5.467891e+06,1253,1K,1011,13.500,01K11025,Inc,Virtual,NaN,...,Virtual South of Paraparaumu Southbound,09 - Wellington,NaN,NaN,NaN,NaN,NaN,NaN,Virtual,13292.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,1.754788e+06,5.443960e+06,1469,59,24,0.115,05940025,Ramp,Continuous,2.6,...,Mungavin Interchange - NB OFF Ramp,09 - Wellington,326.0,10861.0,11015.0,10157.0,11361.0,11071.0,Regional Continuous,10893.000000
1469,1.754787e+06,5.445162e+06,1470,59,23,0.207,05950024,Ramp,Continuous,10.0,...,Porirua Nth Ramp Bridge - NB ON Ramp,09 - Wellington,364.0,10340.0,10501.0,9761.0,10439.0,9618.0,Regional Continuous,10131.800000
1470,1.754812e+06,5.444479e+06,1471,59,24,0.206,05950025,Ramp,Continuous,10.0,...,Mungavin Interchange - NB ON Ramp,09 - Wellington,365.0,5707.0,5778.0,5272.0,5514.0,4432.0,Regional Continuous,5340.600000
1471,1.754843e+06,5.445240e+06,1472,59,23,0.145,05960024,Ramp,Continuous,10.0,...,Porirua Nth Ramp Bridge - SB OFF Ramp,09 - Wellington,349.0,10751.0,10917.0,10118.0,10996.0,9710.0,Regional Continuous,10498.400000


In [104]:
geometry = [Point(xy) for xy in zip(tr_wlg['X'], tr_wlg['Y'])]
wlg_gdf_tr = gpd.GeoDataFrame(tr_wlg, geometry=geometry)
wlg_gdf_tr = wlg_gdf_tr.set_crs(epsg=2193)
wlg_gdf_tr = wlg_gdf_tr.to_crs(epsg=4326)
wlg_gdf_tr

,X,Y,OBJECTID,SH,RS,RP,siteRef,lane,type,percentHeavy,...,region,acceptedDays,AADT5yearsAgo,AADT4yearsAgo,AADT3yearsAgo,AADT2yearsAgo,AADT1yearAgo,siteType,Average AADT,geometry
1248,1.776014e+06,5.477040e+06,1249,1K,1011,0.450,01K11011,Inc,Continuous,0.2,...,09 - Wellington,21.0,8023.0,7885.0,7438.0,7578.0,941.0,Regional Continuous,6373.000000,POINT (175.08773 -40.83880)
1249,1.771258e+06,5.473631e+06,1250,1K,1011,6.500,01K11017,Inc,Continuous,9.7,...,09 - Wellington,204.0,6772.0,7136.0,6880.0,4.0,7400.0,Regional Continuous,5638.400000,POINT (175.03229 -40.87050)
1250,1.770671e+06,5.472273e+06,1251,1K,1011,8.000,01K11019,Inc,Virtual,6.8,...,09 - Wellington,NaN,NaN,NaN,10725.0,11748.0,12482.0,Virtual,11651.666667,POINT (175.02570 -40.88284)
1251,1.768252e+06,5.469309e+06,1252,1K,1011,11.920,01K11023,Inc,Continuous,9.9,...,09 - Wellington,332.0,6312.0,6554.0,5821.0,6366.0,6849.0,Regional Continuous,6380.400000,POINT (174.99780 -40.91003)
1252,1.767713e+06,5.467891e+06,1253,1K,1011,13.500,01K11025,Inc,Virtual,NaN,...,09 - Wellington,NaN,NaN,NaN,NaN,NaN,NaN,Virtual,13292.800000,POINT (174.99180 -40.92291)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,1.754788e+06,5.443960e+06,1469,59,24,0.115,05940025,Ramp,Continuous,2.6,...,09 - Wellington,326.0,10861.0,11015.0,10157.0,11361.0,11071.0,Regional Continuous,10893.000000,POINT (174.84438 -41.14092)
1469,1.754787e+06,5.445162e+06,1470,59,23,0.207,05950024,Ramp,Continuous,10.0,...,09 - Wellington,364.0,10340.0,10501.0,9761.0,10439.0,9618.0,Regional Continuous,10131.800000,POINT (174.84406 -41.13009)
1470,1.754812e+06,5.444479e+06,1471,59,24,0.206,05950025,Ramp,Continuous,10.0,...,09 - Wellington,365.0,5707.0,5778.0,5272.0,5514.0,4432.0,Regional Continuous,5340.600000,POINT (174.84453 -41.13624)
1471,1.754843e+06,5.445240e+06,1472,59,23,0.145,05960024,Ramp,Continuous,10.0,...,09 - Wellington,349.0,10751.0,10917.0,10118.0,10996.0,9710.0,Regional Continuous,10498.400000,POINT (174.84471 -41.12938)


Christchurch

In [100]:
tr_chc = traffic_df[traffic_df['region'] == '11 - Canterbury']
tr_chc['Average AADT'] = tr_chc[['AADT5yearsAgo', 'AADT4yearsAgo', 'AADT3yearsAgo', 'AADT2yearsAgo', 'AADT1yearAgo']].mean(axis=1)
median_average_aadt = tr_chc['Average AADT'].median()
tr_chc['Average AADT'] = tr_chc['Average AADT'].fillna(median_average_aadt)
tr_chc

C:\Users\Alvin Zhao\AppData\Local\Temp\ipykernel_4064\432977496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_chc['Average AADT'] = tr_chc[['AADT5yearsAgo', 'AADT4yearsAgo', 'AADT3yearsAgo', 'AADT2yearsAgo', 'AADT1yearAgo']].mean(axis=1)
C:\Users\Alvin Zhao\AppData\Local\Temp\ipykernel_4064\432977496.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_chc['Average AADT'] = tr_chc['Average AADT'].fillna(median_average_aadt)


,X,Y,OBJECTID,SH,RS,RP,siteRef,lane,type,percentHeavy,...,description,region,acceptedDays,AADT5yearsAgo,AADT4yearsAgo,AADT3yearsAgo,AADT2yearsAgo,AADT1yearAgo,siteType,Average AADT
1522,1.568710e+06,5.188871e+06,1523,1J,332,1.240000,01J00333,Both,Non-Continuous,11.9,...,Christchurch - Johns Rd East of Groynes Entrance,11 - Canterbury,78.0,5230.0,5450.0,5313.0,5571.0,5449.0,Regional Non-Continuous,5402.6
1523,1.660190e+06,5.315468e+06,1524,1S,138,6.260000,01S00144,Both,National,23.3,...,KAIKOURA - Telemetry Site 9 - Nth of Hapuku River,11 - Canterbury,325.0,2870.0,3097.0,2721.0,2637.0,2596.0,Telemetry,2784.2
1524,1.655910e+06,5.306740e+06,1525,1S,155,1.510000,01S00157,Both,Continuous,10.1,...,Kaikoura - Beach Rd,11 - Canterbury,362.0,8443.0,8250.0,4089.0,6552.0,7194.0,Regional Continuous,6905.6
1525,1.652250e+06,5.303932e+06,1526,1S,155,7.460000,01S00163,Both,Continuous,18.8,...,Kaikoura - Nth of Kowhai R.,11 - Canterbury,362.0,4078.0,4104.0,3613.0,3593.0,3669.0,Regional Continuous,3811.4
1526,1.624699e+06,5.272719e+06,1527,1S,195,16.650000,01S00212,Both,Continuous,22.8,...,Parnassus - Nth of Leader R.,11 - Canterbury,362.0,2616.0,2916.0,2612.0,2691.0,2736.0,Regional Continuous,2714.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,1.369431e+06,5.103780e+06,1777,80,0,0.790000,08000001,Both,Continuous,6.1,...,Lake Pukaki - Nth of SH8 Junction,11 - Canterbury,255.0,1262.0,1383.0,835.0,505.0,842.0,Regional Continuous,965.4
1777,1.451675e+06,5.048552e+06,1778,82,0,0.270000,08200000,Both,Continuous,8.2,...,Waimate Turnoff - Sth of SH1 Junction,11 - Canterbury,311.0,1980.0,1992.0,1728.0,1890.0,1795.0,Regional Continuous,1877.0
1778,1.445244e+06,5.044654e+06,1779,82,0,7.920000,08200008,Both,Continuous,5.3,...,Waimate - Queen St Nth-east of William St,11 - Canterbury,304.0,3567.0,3388.0,3574.0,3502.0,3383.0,Regional Continuous,3482.8
1779,1.444078e+06,5.042899e+06,1780,82,0,10.160000,08200010,Both,Continuous,11.4,...,Waimate - 500m west of showgrounds,11 - Canterbury,310.0,1051.0,908.0,1281.0,876.0,886.0,Regional Continuous,1000.4


In [105]:
geometry = [Point(xy) for xy in zip(tr_chc['X'], tr_chc['Y'])]
chc_gdf_tr = gpd.GeoDataFrame(tr_chc, geometry=geometry)
chc_gdf_tr = chc_gdf_tr.set_crs(epsg=2193)
chc_gdf_tr = chc_gdf_tr.to_crs(epsg=4326)
chc_gdf_tr

,X,Y,OBJECTID,SH,RS,RP,siteRef,lane,type,percentHeavy,...,region,acceptedDays,AADT5yearsAgo,AADT4yearsAgo,AADT3yearsAgo,AADT2yearsAgo,AADT1yearAgo,siteType,Average AADT,geometry
1522,1.568710e+06,5.188871e+06,1523,1J,332,1.240000,01J00333,Both,Non-Continuous,11.9,...,11 - Canterbury,78.0,5230.0,5450.0,5313.0,5571.0,5449.0,Regional Non-Continuous,5402.6,POINT (172.61327 -43.45241)
1523,1.660190e+06,5.315468e+06,1524,1S,138,6.260000,01S00144,Both,National,23.3,...,11 - Canterbury,325.0,2870.0,3097.0,2721.0,2637.0,2596.0,Telemetry,2784.2,POINT (173.73034 -42.31069)
1524,1.655910e+06,5.306740e+06,1525,1S,155,1.510000,01S00157,Both,Continuous,10.1,...,11 - Canterbury,362.0,8443.0,8250.0,4089.0,6552.0,7194.0,Regional Continuous,6905.6,POINT (173.67926 -42.38962)
1525,1.652250e+06,5.303932e+06,1526,1S,155,7.460000,01S00163,Both,Continuous,18.8,...,11 - Canterbury,362.0,4078.0,4104.0,3613.0,3593.0,3669.0,Regional Continuous,3811.4,POINT (173.63505 -42.41516)
1526,1.624699e+06,5.272719e+06,1527,1S,195,16.650000,01S00212,Both,Continuous,22.8,...,11 - Canterbury,362.0,2616.0,2916.0,2612.0,2691.0,2736.0,Regional Continuous,2714.2,POINT (173.30155 -42.69762)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,1.369431e+06,5.103780e+06,1777,80,0,0.790000,08000001,Both,Continuous,6.1,...,11 - Canterbury,255.0,1262.0,1383.0,835.0,505.0,842.0,Regional Continuous,965.4,POINT (170.11532 -44.18277)
1777,1.451675e+06,5.048552e+06,1778,82,0,0.270000,08200000,Both,Continuous,8.2,...,11 - Canterbury,311.0,1980.0,1992.0,1728.0,1890.0,1795.0,Regional Continuous,1877.0,POINT (171.12779 -44.70107)
1778,1.445244e+06,5.044654e+06,1779,82,0,7.920000,08200008,Both,Continuous,5.3,...,11 - Canterbury,304.0,3567.0,3388.0,3574.0,3502.0,3383.0,Regional Continuous,3482.8,POINT (171.04548 -44.73477)
1779,1.444078e+06,5.042899e+06,1780,82,0,10.160000,08200010,Both,Continuous,11.4,...,11 - Canterbury,310.0,1051.0,908.0,1281.0,876.0,886.0,Regional Continuous,1000.4,POINT (171.03021 -44.75031)


## Geo-special analysis

### Auckland

In [113]:
map_center = [akl_gdf10.geometry.iloc[0].y, akl_gdf10.geometry.iloc[0].x]
m = folium.Map(location=map_center, zoom_start=12)

lat_lon_concentration = akl_gdf10.apply(lambda row: [row.geometry.y, row.geometry.x, row['Concentration']], axis=1)
max_concentration = max(akl_gdf10['Concentration'])
min_concentration = min(akl_gdf10['Concentration'])
concentration_colormap = cm.linear.YlOrRd_09.scale(min_concentration, max_concentration)
concentration_colormap.caption = 'PM10 Concentration Level'

HeatMap(data=lat_lon_concentration.tolist(), radius=10).add_to(m)

for lat, lon, concentration in lat_lon_concentration:
    folium.CircleMarker(
        location=[lat, lon],
        radius=5 + (concentration - min_concentration) / (max_concentration - min_concentration) * 10,
        popup=f'Concentration: {concentration}',
        color=concentration_colormap(concentration),
        fill=True,
        fill_color=concentration_colormap(concentration)
    ).add_to(m)

lat_lon_AADT = akl_gdf_tr.apply(lambda row: [row.geometry.y, row.geometry.x, row['Average AADT']], axis=1)
max_AADT = max(akl_gdf_tr['Average AADT'])
min_AADT = min(akl_gdf_tr['Average AADT'])

HeatMap(data=lat_lon_AADT.tolist(), radius=10, gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1: 'red'}).add_to(m)

m.add_child(concentration_colormap)

display(m)

### Wellington

In [114]:
map_center = [wlg_gdf10.geometry.iloc[0].y, wlg_gdf10.geometry.iloc[0].x]
m = folium.Map(location=map_center, zoom_start=12)

lat_lon_concentration = wlg_gdf10.apply(lambda row: [row.geometry.y, row.geometry.x, row['Concentration']], axis=1)
max_concentration = max(wlg_gdf10['Concentration'])
min_concentration = min(wlg_gdf10['Concentration'])
concentration_colormap = cm.linear.YlOrRd_09.scale(min_concentration, max_concentration)
concentration_colormap.caption = 'PM10 Concentration Level'

HeatMap(data=lat_lon_concentration.tolist(), radius=10).add_to(m)

for lat, lon, concentration in lat_lon_concentration:
    folium.CircleMarker(
        location=[lat, lon],
        radius=5 + (concentration - min_concentration) / (max_concentration - min_concentration) * 10,
        popup=f'Concentration: {concentration}',
        color=concentration_colormap(concentration),
        fill=True,
        fill_color=concentration_colormap(concentration)
    ).add_to(m)

lat_lon_AADT = wlg_gdf_tr.apply(lambda row: [row.geometry.y, row.geometry.x, row['Average AADT']], axis=1)
max_AADT = max(wlg_gdf_tr['Average AADT'])
min_AADT = min(wlg_gdf_tr['Average AADT'])

HeatMap(data=lat_lon_AADT.tolist(), radius=10, gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1: 'red'}).add_to(m)

m.add_child(concentration_colormap)

display(m)

### Christchurch

In [115]:
map_center = [chc_gdf10.geometry.iloc[0].y, chc_gdf10.geometry.iloc[0].x]
m = folium.Map(location=map_center, zoom_start=12)

lat_lon_concentration = chc_gdf10.apply(lambda row: [row.geometry.y, row.geometry.x, row['Concentration']], axis=1)
max_concentration = max(chc_gdf10['Concentration'])
min_concentration = min(chc_gdf10['Concentration'])
concentration_colormap = cm.linear.YlOrRd_09.scale(min_concentration, max_concentration)
concentration_colormap.caption = 'PM10 Concentration Level'

HeatMap(data=lat_lon_concentration.tolist(), radius=10).add_to(m)

for lat, lon, concentration in lat_lon_concentration:
    folium.CircleMarker(
        location=[lat, lon],
        radius=5 + (concentration - min_concentration) / (max_concentration - min_concentration) * 10,
        popup=f'Concentration: {concentration}',
        color=concentration_colormap(concentration),
        fill=True,
        fill_color=concentration_colormap(concentration)
    ).add_to(m)

lat_lon_AADT = chc_gdf_tr.apply(lambda row: [row.geometry.y, row.geometry.x, row['Average AADT']], axis=1)
max_AADT = max(chc_gdf_tr['Average AADT'])
min_AADT = min(chc_gdf_tr['Average AADT'])

HeatMap(data=lat_lon_AADT.tolist(), radius=10, gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1: 'red'}).add_to(m)

m.add_child(concentration_colormap)

display(m)